In [9]:
import pandas as pd # Pour le dataframe
from skimage.transform import resize
import numpy as np # Pour la normalisation et calculs de moyenne
import matplotlib.pyplot as plt # Pour la visualisation

from PIL import Image

import librosa # Pour l'extraction des features et la lecture des fichiers wav
import librosa.display # Pour récupérer les spectrogrammes des audio
import librosa.feature

import os # C'est ce qui va nous permettre d'itérer sur les fichiers de l'environnement de travail

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve, RandomizedSearchCV # Split de dataset et optimisation des hyperparamètres
from sklearn.ensemble import RandomForestClassifier # Random forest
from sklearn.ensemble import GradientBoostingClassifier # XGBoost
from sklearn.neighbors import KNeighborsClassifier # k-NN
from sklearn.svm import SVC # SVM
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score, zero_one_loss, classification_report # Métriques pour la mesure de performances
from sklearn.preprocessing import normalize, StandardScaler

import tensorflow as tf # Pour le reseau de neurones simple et pour le CNN
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from keras.optimizers import Adam
from keras.utils import to_categorical

from xgboost import XGBClassifier
from pprint import pprint

In [ ]:
#Voici la liste des genres musicaux représentés dans la notre base de données, on en compte 10.
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 
          'jazz', 'metal', 'pop', 'reggae', 'rock']

On commence par initialiser le dictionnaire avec les clés qui sont les noms des genres.\
On met autant de listes vides qu'il n'y a de genres.

In [ ]:
# Créer un dictionnaire vide pour stocker les données audio pour chaque genre
audio_files = {}

# Itérer sur la liste des genres
for g in genres:
    # Créer une liste vide pour stocker les données audio pour le genre actuel
    audio_files[g] = []

In [ ]:
# Itérer sur les fichiers audio dans le répertoire pour le genre actuel
for audio in os.listdir(f'./Data/genres_original/{g}'):
    # Charger les données audio en utilisant librosa et les ajouter à la liste pour le genre actuel
    audio_data = librosa.load(f'./Data/genres_original/{g}/{audio}')[0]
    audio_files[g].append(audio_data)

In [ ]:
print(audio) #Une chaîne de caractère est passée et non un échantillon audio...

In [ ]:
print(audio_data)

Maintenant que nous avons présentés toutes les features que l'on a considéré, construisons un pipeline pour créer une base d'apprentissage.\
Toutes ces features sont facilement calculables avec la librairie Python Librosa.\
_Ce pipeline nous servira aussi pour essayer nos modèles sur des fichiers musicaux externes (YouTube par exemple)._

In [ ]:
def audio_pipeline(audio):
  
  features = []

  # Calcul du ZCR

  zcr = librosa.zero_crossings(audio)
  features.append(sum(zcr))

  # Calcul de la moyenne du Spectral centroid

  spectral_centroids = librosa.feature.spectral_centroid(y=audio)[0]
  features.append(np.mean(spectral_centroids))
  
  # Calcul du spectral rolloff point

  rolloff = librosa.feature.spectral_rolloff(y=audio)
  features.append(np.mean(rolloff))

  # Calcul des moyennes des MFCC

  mfcc = librosa.feature.mfcc(y=audio)

  for x in mfcc:
    features.append(np.mean(x))

  return features

On écrit les noms des features dans les colonnes d'un dataframe :

In [ ]:
column_names = ['zcr', 'spectral_c', 'rolloff', 'mfcc1', 'mfcc2', 'mfcc3',
                'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9',
                'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15',
                'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20', 'label']

In [ ]:
df = pd.DataFrame(columns = column_names)

In [ ]:
df.head()

In [ ]:
i = 0
for g in genres:
  for music in audio_files[g]:
    df.loc[i] = audio_pipeline(music)+[g]
    i+=1

In [ ]:
df.to_csv('./Data/csv_result/result.csv', index = False)

In [ ]:
df = pd.read_csv("./Data/csv_result/result.csv")

In [ ]:
df.head()

In [ ]:
selector = VarianceThreshold(threshold=(0.2))
selected_features = selector.fit_transform(df[['zcr', 'spectral_c', 'rolloff', 'mfcc1', 'mfcc2', 'mfcc3',
                                              'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9',
                                              'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15',
                                              'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20']])

df_select = pd.DataFrame(selected_features)

____

### Corrélations
Bien que les méthodes que nous allons implémenter dans ce projet gérent plutôt bien les variables corrélées, on peut afficher une matrice de corrélation pour mieux comprendre nos données.

In [ ]:
f = plt.figure(figsize=(8, 8))
#ax = plt.gca()
plt.matshow(df_select.corr(), fignum=f.number)
plt.xticks(range(df.shape[1]), df.columns, fontsize=14, rotation=45)
plt.yticks(range(df.shape[1]), df.columns, fontsize=14)
#ax.tick_params(axis="x", bottom=True)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Matrice de corrélation', fontsize=16, y=-0.15)

___

# Introduction aux CNN
L'utilisation de réseaux de neurones convolutionnels représente la meilleure approche lorsqu'il s'agit de faire de la classification d'images.

### Création des images
Dans la cellule ci-dessous, nous assoçions tous les fichiers audio à leur spectrogramme.\
Pour cela nous devons itérer sur ces fichiers, ploter leurs spectrogrammes respectifs et sauvegarder l'image dans un dossier que nous avons créé au préalable.

Tous les spectrogrammes seront ensuite accessibles depuis le dictionnaire audio_files :

In [ ]:
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 
          'jazz', 'metal', 'pop', 'reggae', 'rock']

audio_files = {}

for g in genres:
  audio_files[g] = []

In [ ]:
# Itérer sur les fichiers audio dans le répertoire pour le genre actuel
for g in genres:
  for audio in os.listdir(f'./Data/genres_original/{g}'):
    audio_files[g].append(librosa.core.load(f'./Data/genres_original/{g}/{audio}'))

### Preprocessing
On peut maintenant stocker toutes les images sous forme de matrices numpy dans un vecteur X :

Nous allons maintenant mettre en ordre nos données d'entraînement.\
On doit avoir un array que nous appelerons X (pour ne pas confondre avec le précédent X), constitué de toutes les matrices des images, ainsi qu'un array appelé y_cnn avec tous les labels.

In [ ]:
mel_specs = []

for g in genres:

  for audio in audio_files[g]:

    y = audio[0]
    sr = audio[1]

    spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=1024)
    spect = librosa.power_to_db(spect, ref=np.max)
        
# On modifie la taille des images 128 x 660 en gardant les paramètres proposés dans l'article initial
    if spect.shape[1] != 600:
      # spect.resize(128,660, refcheck=False)
      spect = spect[:128,:600]
      spect.reshape(128,600)

    mel_specs.append(spect)
        
X = np.array(mel_specs)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(mel_specs[0])

On peut maintenant créer le vecteur des labels y_cnn, pour l'instant nos images sont dans l'ordre de la construction de y_cnn est triviale :

In [ ]:
y_cnn = []

for i in range(len(genres)):
  y_cnn +=100*[i] # On a 100 images pour chaque genre
y_cnn = np.array(y_cnn)

Pour pouvoir utiliser y_cnn il faut la transformer en variables catégoriques par un encodage One-Hot, scikit-learn permet de le faire très facilement.

In [ ]:
# On split ensuite nos données en données de test et données d'entraînements :
y_cnn = to_categorical(y_cnn)
x_cnn_train, x_cnn_test, y_cnn_train, y_cnn_test = train_test_split(X, y_cnn)

In [ ]:
# On pense à normaliser les données, c'est important pour l'analyse d'images :
x_cnn_train /= -80
x_cnn_test /= -80

In [ ]:
# A présent on met en forme nos données pour l'entraînement :
x_cnn_train = x_cnn_train.reshape(x_cnn_train.shape[0], 128, 600, 1)
x_cnn_test = x_cnn_test.reshape(x_cnn_test.shape[0], 128, 600, 1)

A ce stade nous avons bien deux arrays X et y_cnn.\
X contient toutes les matrices représentants les images des spectrogrammes et y_cnn contient les labels pour chaque image.

Vérifions les tailles de x_cnn et y_cnn pour être sur qu'il n'y a pas de problème :

In [ ]:
print(x_cnn_train.shape)
print(y_cnn_train.shape)

___

## Construction du CNN

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

In [ ]:
def create_model():
    model = tf.keras.models.Sequential()

    # Première couche Conv2D
    model.add(tf.keras.layers.Conv2D(16, kernel_size=(3, 3),
                    activation='relu',
                    input_shape=(128,600,1)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # Deuxième couche Conv2D
    model.add(tf.keras.layers.Conv2D(32, 
                    kernel_size=(3, 3), 
                    activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # Dropout
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Conv2D(32, 
                     kernel_size=(3, 3), 
                     activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # Aplatir et couches entièrement connectées
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(64, activation='relu')) #Couche enlevée pour améliorer les perfs
    model.add(tf.keras.layers.Dense(32, activation='relu'))

    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    return model


On peut à présent commencer l'entraînement.

On peut regader l'évolution des erreurs et de la loss au fur et à mesure des époques d'entraînements du modèle.

#### Configurer K-Fold :

In [ ]:
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True)

In [ ]:
acc_per_fold = []
loss_per_fold = []
# Liste pour stocker les historiques
all_histories = []

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-6)
callbacks = [early_stopping, reduce_lr]

for train, test in kfold.split(x_cnn_train, y_cnn_train):
    model = create_model()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Entraîner le modèle avec EarlyStopping
    history = model.fit(x_cnn_train[train], y_cnn_train[train], epochs=35, batch_size=64, 
                        validation_data=(x_cnn_test, y_cnn_test), 
                        callbacks=[callbacks])
    
    # Stocker l'historique
    all_histories.append(history.history)
    
    # Évaluation du modèle
    scores = model.evaluate(x_cnn_train[test], y_cnn_train[test], verbose=0)
    print(f'Score for fold {len(acc_per_fold) + 1}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

# Moyenne et écart-type
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


In [ ]:
# Tracer les courbes
# (Moyenner les historiques de tous les plis)
history_to_plot = all_histories[0]  # Utiliser l'historique du premier pli comme exemple

plt.plot(history_to_plot['loss'], label="Train")
plt.plot(history_to_plot['val_loss'], label="Val")
plt.legend(loc='upper left')
plt.title("Loss")
plt.show()

plt.plot(history_to_plot['accuracy'], label="Train")
plt.plot(history_to_plot['val_accuracy'], label="Val")
plt.legend(loc='upper left')
plt.title("Accuracy")
plt.show()

In [ ]:
model.summary()

##### Bilan des performances

In [ ]:
sns.set()
predictions = model.predict(x_cnn_test)
mat = confusion_matrix(np.argmax(y_cnn_test, 1), np.argmax(predictions, 1))
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, xticklabels=genres, yticklabels=genres)
plt.xlabel('true label')
plt.ylabel('predicted label')

___

##### Création de la méthode de prédiction

In [ ]:
model = load_model('./BadBoyModelV3.keras')
audio_file = './Data/genres_original/blues/blues.00000.wav'

In [ ]:
from skimage.transform import resize

def predict_audio_genre(audio_file, model, genres):
    y, sr = librosa.load(audio_file, mono=True)
    # Compute the spectrogram of the audio file
    spectrogram = np.abs(librosa.stft(y))
    # Resize the spectrogram to match the input shape of the model
    input_shape = (128, 660)
    spectrogram = resize(spectrogram, input_shape)
    # Reshape the spectrogram to match the expected input shape of the model
    spectrogram = np.expand_dims(spectrogram, axis=-1)
    spectrogram = np.expand_dims(spectrogram, axis=0)
    # Make a prediction with the preprocessed audio
    predictions = model.predict(spectrogram)
    # Get the predicted class index
    predicted_class_index = np.argmax(predictions[0])
    # Get the name of the predicted class
    predicted_class_name = genres[predicted_class_index]
    
    for i in range(len(genres)):
        actual = predictions[0][i]
        print(round(actual, 2), genres[i])
        
    print(f'The predicted class is: {predicted_class_name}')
    return predicted_class_name

# Example usage
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

predicted_genre = predict_audio_genre(audio_file, model, genres)


1/1 [==============================] - 0s 70ms/step
0.0 blues
0.0 classical
0.0 country
0.0 disco
1.0 hiphop
0.0 jazz
0.0 metal
0.0 pop
0.0 reggae
0.0 rock
The predicted class is: hiphop


### Notes
Les genres les moins biens reconnus sont :
- Le Rock
- Le Reggae
- La pop 
- Le blues

In [ ]:
# model.save('BadBoyModelV3.keras')

In [34]:
model = keras.models.load_model('./BadBoyModel.keras')
audio_file = './Data/genres_original/metal/metal.00000.wav'
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

In [35]:
predicted_genre = predict_audio_genre(audio_file, model, genres)

1/1 [==============================] - 0s 91ms/step
0.0 blues
0.0 classical
0.0 country
0.0 disco
1.0 hiphop
0.0 jazz
0.0 metal
0.0 pop
0.0 reggae
0.0 rock
The predicted class is: hiphop


___

##### Ré-entraînement du modèle 

In [ ]:
model = keras.models.load_model('./BadBoyModelV3.keras')

In [ ]:
second_audio_files = './Data/genres_original/metal/metal.00097.wav'
y, sr = librosa.load(second_audio_files, mono=True)

Créer le spectrogramme :

In [ ]:
spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=1024)
spect = librosa.power_to_db(spect, ref=np.max)
if spect.shape[1] != 600:
    spect = spect[:, :600]
spect = spect[np.newaxis, :, :, np.newaxis]
spect /= -80 # Normaliser le spectrogramme

Redimensionner le spectrogramme à la forme attendue :

In [ ]:
from skimage.transform import resize
spect_resized = resize(spect[0, :, :, 0], (128, 660))
spect_resized = spect_resized[np.newaxis, :, :, np.newaxis]

In [ ]:
genre_index = genres.index('metal')  # Remplacez 'metal' par le genre de votre choix
second_classes = np.array([genre_index])
second_classes = to_categorical(second_classes, num_classes=len(genres))

Ré-entraîner le modèle :

In [ ]:
model.fit(spect_resized, second_classes, epochs=10, batch_size=1)

##### Évaluer le modèle :

**A revoir !**

___

##### Test avec plusieurs prédictions

In [29]:
# Charger le modèle
model = load_model('./BadBoyModel.keras')

# Liste des genres musicaux
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 
          'jazz', 'metal', 'pop', 'reggae', 'rock']

# Dossier contenant les fichiers audio pour le genre "metal"
song_directory = './Data/genres_original/metal/'

# Itérer sur chaque fichier audio dans le dossier "metal"
for filename in os.listdir(song_directory):
    audio_file = os.path.join(song_directory, filename)
    
    print(f"Prediction for {filename}:")
    predicted_genre = predict_audio_genre(audio_file, model, genres)
    print("------------------------------------------------")

Prediction for metal.00000.wav:
1/1 [==============================] - 0s 72ms/step
0.0 blues
0.0 classical
0.0 country
0.0 disco
1.0 hiphop
0.0 jazz
0.0 metal
0.0 pop
0.0 reggae
0.0 rock
The predicted class is: hiphop
------------------------------------------------
Prediction for metal.00001.wav:
1/1 [==============================] - 0s 31ms/step
0.0 blues
0.0 classical
0.0 country
0.0 disco
1.0 hiphop
0.0 jazz
0.0 metal
0.0 pop
0.0 reggae
0.0 rock
The predicted class is: hiphop
------------------------------------------------
Prediction for metal.00002.wav:
1/1 [==============================] - 0s 27ms/step
0.0 blues
0.0 classical
0.0 country
0.0 disco
1.0 hiphop
0.0 jazz
0.0 metal
0.0 pop
0.0 reggae
0.0 rock
The predicted class is: hiphop
------------------------------------------------
Prediction for metal.00003.wav:
1/1 [==============================] - 0s 31ms/step
0.0 blues
0.0 classical
0.0 country
0.0 disco
1.0 hiphop
0.0 jazz
0.0 metal
0.0 pop
0.0 reggae
0.0 rock
The pred